Drive mount 후 working directory 이동 -- 런타임 초기화될때마다(코랩 킬 때마다)



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/kfashion

/content/drive/My Drive/kfashion


mmdetection 설치 -- 런타임 초기화될때마다(코랩 킬 때마다)

In [ ]:
!pip install openmim
!mim install mmdet

baseline code 가져오기 -- 1번만

In [ ]:
git clone 'https://github.com/dacon-ai/K-fashion-baseline.git'

Train/Test 폴더별 이미지 합치기 -- 1번만

In [3]:
from shutil import copyfile
from glob import glob
from tqdm import tqdm
import os

def movefiles(filepath, movepath):
    copyfile(filepath, movepath)
    
train_paths = glob(os.path.join('dataset', 'train', '*', '*.jpg'))
test_paths = glob(os.path.join('dataset', 'test', '*', '*.jpg'))

train_rename = ['dataset/train_all/' + train_path.split('/')[-1] for train_path in train_paths]
test_rename = ['dataset/test_all/' + test_path.split('/')[-1] for test_path in test_paths]

In [ ]:
for i in tqdm(range((61339+14106), len(train_paths))):
  copyfile(train_paths[i], train_rename[i])

 24%|██▍       | 4842/20019 [24:02<1:12:18,  3.50it/s]

In [ ]:
for i in tqdm(range(len(test_paths))): #완료
  copyfile(test_paths[i], test_rename[i])

100%|██████████| 8000/8000 [39:16<00:00,  3.39it/s]


Train/Validation 데이터셋 분할 -- 1번만


In [ ]:
import json
import os
import random


def split_dataset(input_json, input_csv, output_dir, val_ratio, random_seed):
    random.seed(random_seed)

    with open(input_json) as json_reader:
        dataset = json.load(json_reader)

    images = dataset['images']
    annotations = dataset['annotations']
    categories = dataset['categories']

    # file_name에 prefix 디렉토리까지 포함 (CocoDataset 클래스를 사용하는 경우)
    for image in images:
        image['file_name'] = '{}/{}'.format(image['file_name'][0], image['file_name'])

    image_ids = [x.get('id') for x in images]
    image_ids.sort()
    random.shuffle(image_ids)

    num_val = int(len(image_ids) * val_ratio)
    num_train = len(image_ids) - num_val

    image_ids_val, image_ids_train = set(image_ids[:num_val]), set(image_ids[num_val:])

    train_images = [x for x in images if x.get('id') in image_ids_train]
    val_images = [x for x in images if x.get('id') in image_ids_val]
    train_annotations = [x for x in annotations if x.get('image_id') in image_ids_train]
    val_annotations = [x for x in annotations if x.get('image_id') in image_ids_val]

    train_data = {
        'images': train_images,
        'annotations': train_annotations,
        'categories': categories,
    }

    val_data = {
        'images': val_images,
        'annotations': val_annotations,
        'categories': categories,
    }

    output_seed_dir = os.path.join(output_dir, f'seed{random_seed}')
    os.makedirs(output_seed_dir, exist_ok=True)
    output_train_json = os.path.join(output_seed_dir, 'train.json')
    output_val_json = os.path.join(output_seed_dir, 'val.json')
    output_train_csv = os.path.join(output_seed_dir, 'train.csv')
    output_val_csv = os.path.join(output_seed_dir, 'val.csv')

    print(f'write {output_train_json}')
    with open(output_train_json, 'w') as train_writer:
        json.dump(train_data, train_writer)

    print(f'write {output_val_json}')
    with open(output_val_json, 'w') as val_writer:
        json.dump(val_data, val_writer)

    print(f'write {output_train_csv}, {output_val_csv}')
    with open(input_csv, 'r') as csv_reader, \
            open(output_train_csv, 'w') as train_writer, \
            open(output_val_csv, 'w') as val_writer:
        train_writer.write('ImageId,EncodedPixels,Height,Width,CategoryId\n')
        val_writer.write('ImageId,EncodedPixels,Height,Width,CategoryId\n')
        for line in csv_reader:
            if line.startswith('ImageId'): continue
            image_id, encoded_pixels, height, width, category_id = line.strip().split(',')
            image_id = int(image_id)
            if image_id in image_ids_train:
                train_writer.write(line)
            elif image_id in image_ids_val:
                val_writer.write(line)
            else:
                raise ValueError(f'unknown image_id: {image_id}')

In [ ]:
split_dataset(input_json='./dataset/train.json',
              input_csv='./dataset/train.csv',
              output_dir='./dataset/',
              val_ratio=0.1,
              random_seed=13)

write ./dataset/seed13/train.json
write ./dataset/seed13/val.json
write ./dataset/seed13/train.csv, ./dataset/seed13/val.csv


In [ ]:
train_data = json.load(open('./dataset/seed13/train.json'))

print('training data')
print(f'images: {len(train_data["images"])}')
print(f'annotations: {len(train_data["annotations"])}')
print(f'categories: {len(train_data["categories"])}')

training data
images: 85909
annotations: 101403
categories: 21


In [ ]:
val_data = json.load(open('./dataset/seed13/val.json'))

print('validation data')
print(f'images: {len(val_data["images"])}')
print(f'annotations: {len(val_data["annotations"])}')
print(f'categories: {len(val_data["categories"])}')

validation data
images: 9545
annotations: 11265
categories: 21


Train

In [ ]:
cd mmdetection/

/content/drive/My Drive/kfashion/mmdetection


In [ ]:
!python tools/train.py configs/_base_/default_runtime.py 